# Preamble

In [1]:
# Standard libraries
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.rcParams['figure.dpi'] = 300
import random
import csv
import pandas as pd
import h5py
import gc  # garbage collection
# Scikit learn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# PyTorch libraries
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms import ToTensor, Normalize 
# Get dirs
import os
cwd = os.getcwd()# "Code" folder
master_dir = os.path.abspath(os.path.join(cwd, ".."))
## ONNX
# import onnx
# import onnxruntime

When using __Google Colab__, run the following cell

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# code_dir = "/content/drive/MyDrive/KUL/MAI thesis/Code"
# master_dir = os.path.join(code_dir, "..")
# os.chdir(code_dir)
# print(os.getcwd())

In [3]:
# Load own scripts:
%load_ext autoreload
%autoreload 2
import physics
import data
import nnc2p

Load in EOS table:

In [4]:
eos_tables_dir = os.path.join("D:/Coding/Datasets/eos_tables")  # offline
# eos_tables_dir = os.path.join(master_dir, "Data")  # in Google Colab
print(f"Going to look for EOS tables at {eos_tables_dir}")

Going to look for EOS tables at D:/Coding/Datasets/eos_tables


For the training, check if GPU is available (**Google Colab**):

In [5]:
if torch.cuda.is_available(): 
    DEVICE = "cuda:0" 
    torch.set_default_device('cuda')
else: 
    DEVICE = "cpu" 
print(f"Device for training: {DEVICE}")

Device for training: cpu


# Introduction

Garbage collect:

In [6]:
# Browse through objects
for obj in gc.get_objects():
    # see if is an HDF5 file
    if isinstance(obj, h5py.File):
#         print(obj)
        try:
            obj.close()
            del obj
        except:
            pass
gc.collect()

D:\Anaconda3\lib\site-packages\torch\distributed\distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


0

# Exploring EOS tables

In [7]:
# Then specify which we are going to use here
eos_table_filename = "SLy4_0000_rho391_temp163_ye66.h5"

In [50]:
# Open EOS table
eos_table = physics.read_eos_table(os.path.join(eos_tables_dir, eos_table_filename))
# Read in the most important variables and convert them to np arrays
pointsye, pointstemp, pointsrho = eos_table["pointsye"][()][0], eos_table["pointstemp"][()][0], eos_table["pointsrho"][()][0]
dim_ye, dim_temp, dim_rho = pointsye, pointstemp, pointsrho
logrho       = eos_table["logrho"][()]
logtemp      = eos_table["logtemp"][()]
ye           = eos_table["ye"][()]
logpress     = eos_table["logpress"][()]
logenergy    = eos_table["logenergy"][()]
print(np.shape(logenergy))
energy_shift = eos_table["energy_shift"][()][0]
cs2          = eos_table["cs2"][()]
print(f"This EOS table has dimensions (ye, T, rho): {dim_ye} x {dim_temp} x {dim_rho}")
# Small test to see the output of the EOS table
test_ye      = eos_table["ye"][()][0]
test_temp    = eos_table["logtemp"][()][0]
test_rho     = eos_table["logrho"][()][0]
# Get an example output
test_press, test_eps = eos_table["logpress"][()][0, 0, 0], eos_table["logenergy"][()][0, 0, 0]
print(f"Example: ({test_rho}, {test_temp}, {test_ye}) we have (log eps, log p, cs2): ({test_eps}, {test_press}, {cs2[0, 0, 0]}).")
eos_table.close()

(66, 163, 391)
This EOS table has dimensions (ye, T, rho): 66 x 163 x 391
Example: (3.0239960056064277, -3.0, 0.005) we have (log eps, log p, cs2): (19.2791052025363, 17.99956975587081, 1575737840983096.0).


What is the input range of this table?

In [9]:
print(f"logrho    range:({min(logrho)}, {max(logrho)})")
print(f"logtemp range:({min(logtemp)}, {max(logtemp)})")
print(f"ye             range:({min(ye)}, {max(ye)})")

logrho    range:(3.0239960056064277, 16.023996005606428)
logtemp range:(-3.0, 2.4000000000000004)
ye             range:(0.005, 0.655)


See what is inside this EOS table

In [10]:
# # Iterate over keys and save them to list for simplified viewing
# keys = []
# for key in eos_table:
#     keys.append(key)
# print(keys)
# print(len(keys))

## Output
# ['Abar', 'Albar', 'MERGE-space.in', 'MERGE-src.tar.gz', 'MERGE-tables.in', 'MERGE-transition.in', 'SNA-skyrme.in', 'SNA-space.in', 'SNA-src.tar.gz', 'Xa', 'Xh', 'Xl', 'Xn', 'Xp', 'Zbar', 'Zlbar', 'cs2', 'dedt', 'dpderho', 'dpdrhoe', 'energy_shift', 'entropy', 'gamma', 'have_rel_cs2', 'logenergy', 'logpress', 'logrho', 'logtemp', 'meffn', 'meffp', 'mu_e', 'mu_n', 'mu_p', 'muhat', 'munu', 'pointsrho', 'pointstemp', 'pointsye', 'r', 'u', 'ye']

# Construct EOS table for the analytic EOS

Choose the size of the table, by specifying the number of points along one direction, $N$

In [222]:
N = 500

# Choose Nrho, Ntemp, Nye based on this value for N
Nrho = N
Ntemp = N
Nye = N

## Temperature values

These are a bit trickier and have to be obtained from the energy. Use: $ \varepsilon = \frac{1}{\Gamma - 1} N_a k_B T \, . $

For the conversion below, also make sure to see Gmunu, mod_constants.

In [208]:
# Define constants for conversion between eps and temp
gamma = 5/3
N_A = 6.0221367e23

# Cod conversions that are in Gmunu:
k_B = 1.3806488e-16 # erg/K
eps_gf = 1.11265006e-21 # code unit to erg
T_MeV_to_K = 1.1604447522806e10 # K/MeV

## To get to the original version, which gave wrong results in the shocktube, uncomment next line to overrule the conversions
k_B = 1
eps_gf = 1
T_MeV_to_K = 1

# Smallest possible float:
SMALLEST = 2.2250738585072014e-308 

In [209]:
print(eps_gf)
print(T_MeV_to_K)

1
1


In [210]:
def temp_to_eps(temp):
    """Temp is in MeV, so code units"""
    # Get temperature in K
    temp_K = temp * T_MeV_to_K
    # Get eps, in erg
    eps_erg = (1/(gamma - 1)) * N_A * k_B * temp_K
    # Convert erg to code unit
    eps = eps_erg * eps_gf
    return eps

def eps_to_temp(eps):
    """Eps is in code unit"""
    # Convert to erg
    eps_erg = eps/eps_gf
    # Get temperature in Kelvin
    temp_K = ((gamma - 1)/(N_A * k_B)) * eps_erg
    # Convert to MeV
    temp = temp_K/T_MeV_to_K
    return temp

In [211]:
# Take a random test case, see if both transformations match
eps_test = temp_to_eps(5)
eps_to_temp(eps_test)

5.0

What are the ranges of our T table, if we have ranges of eps?

In [212]:
# Get new eps min, for the log:
eps_min = physics.EPS_MIN
eps_max = physics.EPS_MAX
print((eps_min, eps_max))

(0, 2.02)


In [213]:
temp_min = eps_to_temp(physics.EPS_MIN)
# Adjust such that log is well defined
temp_min += SMALLEST
temp_max = eps_to_temp(physics.EPS_MAX)

print((temp_min, temp_max))

(2.2250738585072014e-308, 2.2361941180555848e-24)


Now take the log

In [193]:
logtemp_min = np.log10(temp_min)
logtemp_max = np.log10(temp_max)

print((logtemp_min, logtemp_max))

(-307.6526555685888, -23.650490499201915)


In [223]:
logtemp = np.linspace(logtemp_min, logtemp_max, Ntemp)

## Rho values:

In [224]:
rho_min = physics.RHO_MIN
# Again, add small float for the log
rho_min += SMALLEST
rho_max = physics.RHO_MAX
print((rho_min, rho_max))

(2.2250738585072014e-308, 10.1)


Get log of range

In [225]:
logrho_min = np.log10(rho_min)
logrho_max = np.log10(rho_max)
print((logrho_min, logrho_max))

(-307.6526555685888, 1.0043213737826426)


Get the values

In [226]:
logrho = np.linspace(logrho_min, logrho_max, Nrho)

## Ye

These are dummy values and linear, not log

In [228]:
ye_min = 0
ye_max = 1
ye = np.linspace(ye_min, ye_max, Nye)

In [229]:
ye

array([0.        , 0.00200401, 0.00400802, 0.00601202, 0.00801603,
       0.01002004, 0.01202405, 0.01402806, 0.01603206, 0.01803607,
       0.02004008, 0.02204409, 0.0240481 , 0.0260521 , 0.02805611,
       0.03006012, 0.03206413, 0.03406814, 0.03607214, 0.03807615,
       0.04008016, 0.04208417, 0.04408818, 0.04609218, 0.04809619,
       0.0501002 , 0.05210421, 0.05410822, 0.05611222, 0.05811623,
       0.06012024, 0.06212425, 0.06412826, 0.06613226, 0.06813627,
       0.07014028, 0.07214429, 0.0741483 , 0.0761523 , 0.07815631,
       0.08016032, 0.08216433, 0.08416834, 0.08617234, 0.08817635,
       0.09018036, 0.09218437, 0.09418838, 0.09619238, 0.09819639,
       0.1002004 , 0.10220441, 0.10420842, 0.10621242, 0.10821643,
       0.11022044, 0.11222445, 0.11422846, 0.11623246, 0.11823647,
       0.12024048, 0.12224449, 0.1242485 , 0.12625251, 0.12825651,
       0.13026052, 0.13226453, 0.13426854, 0.13627255, 0.13827655,
       0.14028056, 0.14228457, 0.14428858, 0.14629259, 0.14829

## Columns

The "columns" represent the output variables, such as $p$, $\varepsilon$ and $c_s^2$ most importantly

Auxiliary function: derivatives of $p$ wrt $\rho$ and $\varepsilon$

In [230]:
def chi(eps):
    return (gamma - 1) * eps

def kappa(rho):
    return (gamma - 1) * rho

More information: see appendix of Dieselhorst paper for precise equations!

In [231]:
def get_cs2(prs, rho, eps):
    chi_val = chi(eps)
    kappa_val = kappa(rho)
    if rho == 0:
        rho += SMALLEST
    h = 1 + eps + prs/rho
    rho_sqr = rho ** 2
    if rho_sqr == 0:
        rho_sqr += SMALLEST
    
    return (1/h) * (chi_val + (prs/rho_sqr) * kappa_val)

In [232]:
def get_columns(ye, logtemp, logrho, Nrho = 500, Ntemp = 500, Nye = 500):
    # Initialize as empty
    logpress  = np.empty((Nye, Ntemp, Nrho))
    logenergy = np.empty((Nye, Ntemp, Nrho))
    cs2       = np.empty((Nye, Ntemp, Nrho))
    
    # Get an empty matrix, we copy that one for each ye value to speed up
    logpress_matrix = np.empty((Ntemp, Nrho))
    logenergy_matrix = np.empty((Ntemp, Nrho))
    cs2_matrix = np.empty((Ntemp, Nrho))
    
    # Get the individual values
    for j, logtemp_val in enumerate(logtemp):
        temp_val = 10 ** logtemp_val
        # Compute eps
        eps_val = temp_to_eps(temp_val)
        # Convert to logeps val
        logenergy_val = np.log10(eps_val)
        for k, logrho_val in enumerate(logrho):
            logenergy_matrix[j, k] = logenergy_val

            rho_val = 10 ** logrho_val
            # Compute the pressure
            prs_val = (gamma - 1) * rho_val * eps_val
            # Convert to logpress val
            if prs_val == 0:
                prs_val += SMALLEST
            logpress_val = np.log10(prs_val)
            logpress_matrix[j, k] = logpress_val

            # Compute cs2
            cs2_val = get_cs2(prs_val, rho_val, eps_val)
            cs2_matrix[j, k] = cs2_val
    
    # Copy the matrix for each entry of ye, since ye is "ignored" here
    for i, ye_val in enumerate(ye):
        logpress[i, :, :]  = logpress_matrix
        logenergy[i, :, :] = logenergy_matrix
        cs2[i, :, :]       = cs2_matrix
                
    return logpress, logenergy, cs2

In [233]:
logpress, logenergy, cs2 = get_columns(ye, logtemp, logrho)

### Sanity checks

In [234]:
print(10 ** np.max(logpress))
max_prs = (gamma - 1) * rho_max * eps_max
print(max_prs)

13.601333333333336
13.601333333333335


In [235]:
print(logenergy[0])
print(logenergy[1])

[[-2.83696814e+02 -2.83696814e+02 -2.83696814e+02 ... -2.83696814e+02
  -2.83696814e+02 -2.83696814e+02]
 [-2.83127671e+02 -2.83127671e+02 -2.83127671e+02 ... -2.83127671e+02
  -2.83127671e+02 -2.83127671e+02]
 [-2.82558528e+02 -2.82558528e+02 -2.82558528e+02 ... -2.82558528e+02
  -2.82558528e+02 -2.82558528e+02]
 ...
 [-8.32933861e-01 -8.32933861e-01 -8.32933861e-01 ... -8.32933861e-01
  -8.32933861e-01 -8.32933861e-01]
 [-2.63791246e-01 -2.63791246e-01 -2.63791246e-01 ... -2.63791246e-01
  -2.63791246e-01 -2.63791246e-01]
 [ 3.05351369e-01  3.05351369e-01  3.05351369e-01 ...  3.05351369e-01
   3.05351369e-01  3.05351369e-01]]
[[-2.83696814e+02 -2.83696814e+02 -2.83696814e+02 ... -2.83696814e+02
  -2.83696814e+02 -2.83696814e+02]
 [-2.83127671e+02 -2.83127671e+02 -2.83127671e+02 ... -2.83127671e+02
  -2.83127671e+02 -2.83127671e+02]
 [-2.82558528e+02 -2.82558528e+02 -2.82558528e+02 ... -2.82558528e+02
  -2.82558528e+02 -2.82558528e+02]
 ...
 [-8.32933861e-01 -8.32933861e-01 -8.3293386

In [236]:
print(10**np.min(logenergy))
print(10**np.max(logenergy))

2.0099548415289234e-284
2.0199999999999996


Watch out: there were NaNs in the $c_s^2$ table! Everything fine now? __YES__

In [238]:
# test_array = cs2.flatten()
# np.nanmax(test_array[test_array != float('inf')])
# np.max(test_array)

## To HDF5

In [240]:
mock_table_name = f"analytic_eos_{N}.h5"
with h5py.File(os.path.join(eos_tables_dir, mock_table_name), 'w') as f:
    # Save scalars, but they have to be arrays
    pointsrho_array = [Nrho]
    pointstemp_array = [Ntemp]
    pointsye_array = [Nye]
    dataset = f.create_dataset("pointsrho", data=pointsrho_array)
    dataset = f.create_dataset("pointstemp", data=pointstemp_array)
    dataset = f.create_dataset("pointsye", data=pointsye_array)
    # Save entries
    dataset = f.create_dataset("logrho", data=logrho)
    dataset = f.create_dataset("logtemp", data=logtemp)
    dataset = f.create_dataset("ye", data=ye)
    # Columns
    dataset = f.create_dataset("logenergy", data=logenergy)
    dataset = f.create_dataset("logpress", data=logpress)
    dataset = f.create_dataset("cs2", data=cs2)

## Check the HDF5 file

In [186]:
name = "analytic_eos.h5"
# Open EOS table
eos_table = physics.read_eos_table(os.path.join(eos_tables_dir, name))
# Read in the most important variables and convert them to np arrays
print(eos_table["pointsye"])
pointsye, pointstemp, pointsrho = eos_table["pointsye"][()][0], eos_table["pointstemp"][()][0], eos_table["pointsrho"][()][0]
logrho       = eos_table["logrho"][()]
logtemp      = eos_table["logtemp"][()]
ye           = eos_table["ye"][()]
logpress     = eos_table["logpress"][()]
logenergy    = eos_table["logenergy"][()]
# energy_shift = eos_table["energy_shift"][()][0]
cs2          = eos_table["cs2"][()]
print(f"This EOS table has dimensions (ye, T, rho): {dim_ye} x {dim_temp} x {dim_rho}")
# Small test to see the output of the EOS table
test_ye      = eos_table["ye"][()][-1]
test_temp    = eos_table["logtemp"][()][-1]
test_rho     = eos_table["logrho"][()][-1]
# Get an example output
test_press, test_eps, test_cs2 = eos_table["logpress"][()][-1, -1, -1], eos_table["logenergy"][()][-1, -1, -1], cs2[0, 0, 0]
print(f"Example: ({test_rho}, {test_temp}, {test_ye}) we have (log eps, log p, cs2): ({test_eps}, {test_press}, {test_cs2}).")
print(f"So: (eps, p, cs2): ({10**test_eps}, {10**test_press}, {test_cs2}).")
eos_table.close()

<HDF5 dataset "pointsye": shape (1,), type "<i4">
This EOS table has dimensions (ye, T, rho): 66 x 163 x 391
Example: (1.0043213737826426, 3.0984432164282283, 0.5) we have (log eps, log p, cs2): (0.30535136944662405, 1.1335814841735854, 7.428856353875367e-309).
So: (eps, p, cs2): (2.0200000000000014, 13.601333333333342, 7.428856353875367e-309).


In [186]:
# name = "analytic_eos.h5"
# # Open EOS table
# eos_table = physics.read_eos_table(os.path.join(eos_tables_dir, name))
# # Iterate over keys and save them to list for simplified viewing
# keys = []
# for key in eos_table:
#     keys.append(key)
# print(keys)
# eos_table.close()

['cs2', 'logenergy', 'logpress', 'logrho', 'logtemp', 'pointsrho', 'pointstemp', 'pointsye', 'ye']
9


# What should be the size of the EOS table?

Using the same N for rho, temp or ye is not realistic. Check what the ratios should be based on the tables that are on stellarcollapse

In [ ]:
pointsrho_list